# 0- Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
licenca ="default"

In [3]:
# Google Colab & Kaggle integration
try:
    from amplpy import AMPL, tools
except:
    !pip install -q amplpy
    from amplpy import AMPL, tools
else:
    ampl = tools.ampl_notebook(
        modules=["highs", "gokestrel", "gurobi", "cplex"], # modules to install
        license_uuid=licenca, # license to use
        g=globals()) # instantiate AMPL object and register magics

# 1- Base

Importando a base pré-processada com o limite de produção diário de:
<ul>
    <li>120.000 peças para o “Tipo de buffer” CX e </li>
    <li>100.000 peças para o “Tipo de buffer” EN.</li>
 </ul>

In [4]:
df_process = pd.read_excel('../../01-Data/ordens_pre_process.xlsx')

df_process

,Id,Ordem de produção,Data da ordem de produção,Produto,Comprimento unit,Largura unit,Altura unit,Peso unit,Peças,Caixa padrão,...,Item pai,Cor,Tamanho,Inner,Rota,Capacidade,pai-cor,Volume unit,Volume total,Peso_total
0,0,2,2023-11-14,828646451,284,131,50,0.250,3,CARTON_M,...,2,2,2,1,11,2.720000e+10,2-2,1860200,5580600,0.750
1,1,3,2023-11-09,605410977,250,100,14,0.120,1,CARTON_M,...,3,3,3,1,5,2.720000e+10,3-3,350000,350000,0.120
2,2,7,2023-11-11,545612218,81,87,162,0.640,1,CARTON_S,...,7,0,7,1,1,inf,7-0,1141614,1141614,0.640
3,3,8,2023-11-24,861652334,248,216,128,0.300,5,CARTON_M,...,8,7,5,1,3,inf,8-7,6856704,34283520,1.500
4,4,9,2023-11-13,879695703,393,295,32,0.520,2,CARTON_M,...,9,8,8,1,11,2.720000e+10,9-8,3709920,7419840,1.040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58453,58453,68,2023-11-07,843672948,485,377,30,0.233,3,CARTON_M,...,353,4,1,1,1,inf,353-4,5485350,16456050,0.699
58454,58454,8926,2023-11-23,796269830,443,268,19,0.140,4,CARTON_M,...,958,2,14,1,4,inf,958-2,2255756,9023024,0.560
58455,58455,257,2023-11-07,611398476,412,281,27,0.440,1,CARTON_M,...,554,22,1,1,3,inf,554-22,3125844,3125844,0.440
58456,58456,8936,2023-11-24,882144507,387,283,18,0.107,4,CARTON_M,...,130,9,115,1,3,inf,130-9,1971378,7885512,0.428


Remover os espaços no nome das variáveis

In [5]:
df_process.columns = df_process.columns.str.replace(' ', '_')

In [6]:
df_process.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade', 'pai-cor',
       'Volume_unit', 'Volume_total', 'Peso_total'],
      dtype='object')

In [7]:
df_process[['Loja','Tipo_de_buffer','Classe_de_onda']] = df_process[['Loja','Tipo_de_buffer', 'Classe_de_onda']].astype('string')

In [8]:
df_process['chave_loja_buffer_onda'] = df_process['Loja'] + '_' + df_process['Tipo_de_buffer'] + '_' + df_process['Classe_de_onda']

In [9]:
df_process.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58458 entries, 0 to 58457
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Id                         58458 non-null  int64         
 1   Ordem_de_produção          58458 non-null  int64         
 2   Data_da_ordem_de_produção  58458 non-null  datetime64[ns]
 3   Produto                    58458 non-null  int64         
 4   Comprimento_unit           58458 non-null  int64         
 5   Largura_unit               58458 non-null  int64         
 6   Altura_unit                58458 non-null  int64         
 7   Peso_unit                  58458 non-null  float64       
 8   Peças                      58458 non-null  int64         
 9   Caixa_padrão               58458 non-null  object        
 10  Loja                       58458 non-null  string        
 11  Classe_de_onda             58458 non-null  string        
 12  Tipo

## Caixa padrão

In [10]:
df_caixas_tipo = pd.read_excel('../../01-Data/Dados - desafio disciplina RPVMM.xlsx', sheet_name='Tipos de caixa')
df_caixas_tipo

,Tipo,Tipo de buffer,Comprimento,Largura,Altura,Fator de ocupação,Peças max,Peso max (kg)
0,Caixa 1,CX,380,285,285,0.9,170,23
1,Caixa 2,CX,586,436,272,1.2,170,23
2,Caixa 3,CX,728,427,270,0.9,170,23
3,Caixa 1,EN,380,285,285,0.9,512,23
4,Caixa 2,EN,586,436,272,1.2,512,23
5,Caixa 3,EN,728,427,270,0.9,512,23


In [11]:
df_caixas_tipo.columns = df_caixas_tipo.columns.str.replace(' ', '_')

In [12]:
df_caixas_tipo['volume_util_caixa'] = df_caixas_tipo['Comprimento'] * df_caixas_tipo['Largura'] * df_caixas_tipo['Altura'] * df_caixas_tipo['Fator_de_ocupação']

df_caixas_tipo

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
0,Caixa 1,CX,380,285,285,0.9,170,23,27778950.0
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
2,Caixa 3,CX,728,427,270,0.9,170,23,75538008.0
3,Caixa 1,EN,380,285,285,0.9,512,23,27778950.0
4,Caixa 2,EN,586,436,272,1.2,512,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


Temos um tipo de caixa padrão para cada tipo de buffer

In [13]:
df_caixas_padrao = df_caixas_tipo.loc[[1, 5]]
df_caixas_padrao

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


# 2- Modelagem

## 2.1- CSP com problemas do negócio - múltiplas chaves

Modelagem matemática 

Dado que temos $n$ tipos de itens $i$ para empacotar, para $k$ lojas, cada um com peso $w_{i}$, um volume $v_{i}$, um múltiplo $inner_{i k}$ da quantiade de itens a serem selecioner uma demanda $d_{i}$, para todo $i \in \{1,...,n\}$, e um número $m$ de mochilas (caixas) idênticas com capacidade volumétrica $cv$, uma capacidade de peso $cp$ e uma capacidade em número de peças $np$, então temos:

$$
\begin{aligned}
\text{minimizar} \quad &\sum_{j = 1}^{m} \sum_{k = 1}^{r} y_{j k} \\
\text{sujeito a} \quad &\sum_{i = 1}^{n} w_{i k} \xi_{i j k}*inner_{i k} \leq cp*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\}\\
&\sum_{i = 1}^{n} v_{i k} \xi_{i j k} *inner_{i k} \leq cv*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{i = 1}^{n} \xi_{i j k} *inner_{i k} \leq np*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{j = 1}^{m} \xi_{i j k} *inner_{i k} = d_{i k} \quad \forall i \in \{1,...,n\}, k \in \{1,...,r\}\\
&\xi_{i j k} \geq0 \in \mathbb{Z} \quad \forall i \in \{1,...,n\}, j \in \{1,...,m\} , k \in \{1,...,r\} \\
&y_{j k} \in \{0,1\} \quad \forall j \in \{1,...,m\}, k \in \{1,...,r\}
\end{aligned}
$$


 Seja $\quad$ $\xi_{i j k}$= número de itens tipo $i$ adicionados na mochila $j$, para a loja $k$.


In [49]:
%%ampl_eval
reset;

In [72]:
%%ampl_eval
reset;

param n;
param cv;
param cp;
param np;
param r;

suffix master IN, binary;
suffix block IN, integer;

set I = 1..n ordered;
set K = 1..r ordered;
param w {I,K} >= 0;
param v {I,K} >= 0;
param inner{I,K} >= 0;
param d {I,K} >= 0;

param maxVal := max {i in I, k in K} w[i,k];

param maxVal_peso := max {i in I, k in K} w[i,k];
#param maxVal_volume := max {i in I, k in K} v[i,k];
#param pecas_por_caixa_peso := floor(cp / maxVal_peso);
#param pecas_por_caixa_volume := floor(cv / maxVal_volume);
#param pecas_caixa := min(pecas_por_caixa_peso, pecas_por_caixa_volume);
param pecas_caixa:= floor(cp / maxVal_peso);


param soma_coluna {k in K} := sum{i in I} d[i,k];
#param maior_soma := max{k in K} soma_coluna[k];
param soma := sum{i in I, k in K} d[i, k];
param maxbins := ceil(soma / pecas_caixa);

set J = 1..maxbins ordered;

var e {I,J,K} integer >= 0;
var y {J,K} binary;

minimize FO:  sum {j in J, k in K} y[j,k];

s.t. R1_capacidade_mochila_peso {j in J, k in K}:
   sum {i in I} w[i,k] * e[i,j,k] * inner[i,k] <= cp * y[j,k] suffix block j;

s.t. R2_capacidade_mochila_volume {j in J, k in K}:
   sum {i in I} v[i,k] * e[i,j,k] * inner[i,k] <= cv * y[j,k] suffix block j;

s.t. R3_capacidade_mochila_pecas {j in J, k in K}:
   sum {i in I} e[i,j,k] * inner[i,k] <= np * y[j,k] suffix block j;

s.t. R4_quantidade_demanda {i in I, k in K}:
   sum {j in J} e[i,j,k]* inner[i,k] = d[i,k] suffix master 1;

### 2.1.1- Importar dados

In [73]:
df_process.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade', 'pai-cor',
       'Volume_unit', 'Volume_total', 'Peso_total', 'chave_loja_buffer_onda'],
      dtype='object')

In [74]:
df_process.query('chave_loja_buffer_onda in ("16_EN_3","9_EN_4","33_EN_3")').groupby(['chave_loja_buffer_onda']).agg({'Peso_unit':'sum', 'Volume_unit':'sum', 'Peças':'sum'})\
                                                                                                                .sort_values(by='Peças', ascending=False)

,Peso_unit,Volume_unit,Peças
chave_loja_buffer_onda,,,
16_EN_3,60.809,930529268,1636
33_EN_3,74.834,1151591578,1335
9_EN_4,110.445,1644530130,1297


In [75]:
df_process_situacao1 = df_process.query('chave_loja_buffer_onda in ["16_EN_3","9_EN_4","33_EN_3"]')

df_process_situacao1

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Cor,Tamanho,Inner,Rota,Capacidade,pai-cor,Volume_unit,Volume_total,Peso_total,chave_loja_buffer_onda
31530,31530,42,2023-11-17,882146001,447,337,22,0.300,2,CARTON_M,...,27,6,1,1,inf,42-27,3314058,6628116,0.600,9_EN_4
31544,31544,68,2023-11-14,597596512,461,231,124,0.660,2,CARTON_M,...,22,13,1,1,inf,68-22,13204884,26409768,1.320,33_EN_3
31547,31547,77,2023-11-16,806471223,509,297,25,0.470,8,CARTON_M,...,4,13,1,1,inf,77-4,3779325,30234600,3.760,33_EN_3
31594,31594,105,2023-11-14,553139601,439,277,24,0.208,1,CARTON_M,...,54,1,1,1,inf,177-54,2918472,2918472,0.208,9_EN_4
31598,31598,185,2023-11-16,879651866,450,300,25,0.310,11,CARTON_M,...,50,1,1,1,inf,185-50,3375000,37125000,3.410,16_EN_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58404,58404,8451,2023-11-22,847852356,524,313,67,0.375,10,CARTON_M,...,18,5,1,1,inf,1714-18,10988804,109888040,3.750,9_EN_4
58406,58406,1554,2023-11-07,550056560,524,268,20,0.238,2,CARTON_M,...,4,5,1,1,inf,290-4,2808640,5617280,0.476,16_EN_3
58412,58412,8538,2023-11-16,834665720,414,259,46,0.300,3,CARTON_M,...,274,1,1,1,inf,1820-274,4932396,14797188,0.900,9_EN_4
58422,58422,8595,2023-11-17,883246441,448,355,28,0.320,11,CARTON_M,...,339,5,1,1,inf,954-339,4453120,48984320,3.520,9_EN_4


### 2.1.2- Entrada de dados no modelo

Matriz peso unitário

In [76]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao1['item_loja'] = df_process_situacao1.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,16_EN_3,33_EN_3,9_EN_4
0,1,0.31,0.66,0.300
1,2,0.14,0.47,0.208
2,3,0.18,0.65,0.253
3,4,0.19,0.18,0.390
4,5,0.53,0.25,0.260
...,...,...,...,...
330,331,0.00,0.00,0.293
331,332,0.00,0.00,0.375
332,333,0.00,0.00,0.300
333,334,0.00,0.00,0.320


Matriz volume unitário

In [77]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao1['item_loja'] = df_process_situacao1.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,16_EN_3,33_EN_3,9_EN_4
0,1,3375000.0,13204884.0,3314058.0
1,2,3373608.0,3779325.0,2918472.0
2,3,2903580.0,12661800.0,4718000.0
3,4,2947287.0,3128300.0,7765314.0
4,5,9139475.0,2776813.0,3341250.0
...,...,...,...,...
330,331,0.0,0.0,5861124.0
331,332,0.0,0.0,10988804.0
332,333,0.0,0.0,4932396.0
333,334,0.0,0.0,4453120.0


Matriz inner

In [78]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao1['item_loja'] = df_process_situacao1.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,16_EN_3,33_EN_3,9_EN_4
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,1.0,1.0,1.0
4,5,1.0,1.0,1.0
...,...,...,...,...
330,331,0.0,0.0,1.0
331,332,0.0,0.0,1.0
332,333,0.0,0.0,1.0
333,334,0.0,0.0,1.0


Matriz demanda

In [79]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao1['item_loja'] = df_process_situacao1.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,16_EN_3,33_EN_3,9_EN_4
0,1,11.0,2.0,2.0
1,2,3.0,8.0,1.0
2,3,3.0,12.0,1.0
3,4,17.0,3.0,2.0
4,5,13.0,2.0,3.0
...,...,...,...,...
330,331,0.0,0.0,1.0
331,332,0.0,0.0,10.0
332,333,0.0,0.0,3.0
333,334,0.0,0.0,11.0


In [80]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

In [81]:
buffer = df_process_situacao1['Tipo_de_buffer'].unique()[0]
cp = df_caixas_padrao.query(f'Tipo_de_buffer == "{buffer}"')['Peso_max_(kg)'].values
cv = df_caixas_padrao.query(f'Tipo_de_buffer == "{buffer}"')['volume_util_caixa'].values
np = df_caixas_padrao.query(f'Tipo_de_buffer == "{buffer}"')['Peças_max'].values

print(cp, cv, np)

[23] [75538008.] [512]


#### Parâmetros

In [82]:
ampl.param["n"] = len(pecas)   # tamanho da loa que tem mais itens

ampl.param["cp"] = cp
ampl.param["cv"] = cv
ampl.param["np"] = np
ampl.param["r"] = len(volume.columns) -1

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

In [84]:
ampl.getValue('soma')

4268

In [85]:
ampl.getValue('maxbins')

214

In [86]:
ampl.getValue('pecas_caixa')

20

### 2.1.3- Solver

In [87]:
ampl.option["solver"] = "gcg"
ampl.option["gcg_options"] = "outlev=1 timelimit=200"
ampl.solve()

GCG 4.0.0:   tech:outlev = 1
  lim:time = 200
 added complete decomp for original problem with 214 blocks and 797 masterconss, 0 linkingvars, 0 mastervars, and max white score of   0.704003 
presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 189 upgd conss, 0 impls, 189 clqs
   (1.0s) probing: 51/41088 (0.1%) - 0 fixings, 0 aggregations, 0 implications, 0 bound changes
   (1.0s) probing aborted: 50/50 successive totally useless probings
presolving (2 rounds: 2 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 189 cliques
presolved problem has 171200 variables (41088 bin, 130112 int, 0 impl, 0 cont) and 2723 constraints
    189 constraints of type <setppc>
   2534 constraints of type <linear>
transformed objective value is always integral (scale: 1)
Presolving Time: 2.00
 calculated translation

In [88]:
%%ampl_eval
display {j in J, k in K} y[j,k];

y[j,k] [*,*]
:     1   2   3    :=
1     1   1   1
2     1   1   1
3     1   1   1
4     1   1   1
5     1   1   1
6     1   1   1
7     1   1   1
8     1   1   1
9     1   1   1
10    1   1   1
11    1   1   1
12    1   1   1
13    1   1   1
14    1   1   1
15    1   1   1
16    1   1   1
17    1   1   1
18    1   1   1
19    1   1   1
20    1   1   1
21    1   1   1
22    1   1   1
23    1   1   1
24    1   1   1
25    1   1   1
26    1   1   1
27    1   1   1
28    1   1   1
29    1   1   1
30    1   1   1
31    1   1   1
32    1   1   1
33    1   1   1
34    1   1   1
35    1   1   1
36    1   1   1
37    1   1   1
38    1   1   1
39    1   1   1
40    1   1   1
41    1   1   1
42    1   1   1
43    1   1   1
44    1   1   1
45    1   1   1
46    1   1   1
47    1   1   1
48    1   1   1
49    1   1   1
50    1   1   1
51    1   1   1
52    1   1   1
53    1   1   1
54    0   1   1
55    0   1   1
56    0   1   1
57    0   0   1
58    0   0   1
59    0   0   1
60    0   0   1
61   